---
title: 👩‍❤️‍💋‍👨Code-4-Deep-Learning
password: ""
tags:
  - 人工智能
  - 深度学习
  - python
katex: false
comments: true
aside: true
date: 2022-04-21 00:26:11
cover: https://www.helloimg.com/images/2022/04/21/RHe8xK.png
top_img:
---

<!--
 * @?: *********************************************************************
 * @Author: Weidows
 * @LastEditors: Weidows
 * @LastEditTime: 2022-04-20 23:11:24
 * @FilePath: \Blog-private\scaffolds\post.md
 * @Description:
 * @!: *********************************************************************
-->

## 序

此文为其他文章的代码部分:

> [⚡再啃-Deep-Learning](../../../python/AI/DL)

也提供了 notebook 形式: [代码地址](https://github.com/Weidows-projects/public-post/blob/main/notebook/DL/DL.ipynb)

<a>![分割线](https://fastly.jsdelivr.net/gh/Weidows/Images/img/divider.png)</a>


## 神经网络

### 感知器


In [10]:
def AND(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.7
    tmp = x1 * w1 + x2 * w2
    if tmp <= theta:
        return 0
    else:
        return 1


def OR(x1, x2):
    w1, w2, theta = 0.5, 0.5, 0.2
    tmp = x1 * w1 + x2 * w2
    if tmp <= theta:
        return 0
    else:
        return 1


# 非门只取一个输入,另一个不管
def NOT(x1, x2):
    w1, w2, theta = -1, 0, 0
    tmp = x1 * w1 + x2 * w2 + 1
    return tmp


# 异或门是非线性运算, 需要多层感知器组合
def XOR(x1, x2):
    # 异或门 = (与非门 与 或门)
    return AND(OR(x1, x2), not AND(x1, x2))


print(AND(0, 1), AND(1, 1), OR(0, 1), OR(0, 0))
print(NOT(0, 1), NOT(1, 1), NOT(0, 0), NOT(1, 0))
print(XOR(0, 1), XOR(1, 1), XOR(0, 0), XOR(1, 0))


0 1 1 0
1 0 1 0
1 0 0 1


## OpenMMLab

### MMDetection

#### 装环境

> 从安装到放弃 ( <sup id='cite_ref-1'>[\[1\]](#cite_note-1)</sup> \
> 跟着这篇装的环境:<sup id='cite_ref-2'>[\[2\]](#cite_note-2)</sup>, 官方 document 并不是很全面: <sup id='cite_ref-3'>[\[3\]](#cite_note-3)</sup>

预先装上 anaconda, cuda, cudnn

依赖链: `cuda -> pytorch -> mmcv-full -> mmdet`

每一步依赖前面环境的版本, 即使后面能装上也可能不适配, 任何一步有问题都 can't-run

![](https://www.helloimg.com/images/2022/06/06/ZtZpOm.png)

一眼就能看出来 cuda10.2 最稳

<a>![分割线](https://fastly.jsdelivr.net/gh/Weidows/Images/img/divider.png)</a>


In [1]:
!conda create -n openmmlab -y
!conda activate openmmlab
!conda init

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/openmmlab



Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate openmmlab
#
# To deactivate an active environment, use
#
#     $ conda deactivate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


no change     /opt/anaconda/condabin/conda
no change     /opt/anaconda/bin/conda
no change     /opt/anaconda/bin/conda-env
no change     /opt/anaconda/bin/activate
no change     /opt/anaconda/bin/deactivate
no change     /opt/anaconda/etc/profile.d/conda.sh
no change     /op

In [5]:
!python -m pip install --upgrade pip

# 注意对应机子配置: https://pytorch.org/
%conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

# 与pip/conda同级的专门给mm-lab用的包管理器, 格外鸡肋
%pip install openmim
# %mim install mmdet

# 歇逼了, 官方没给windows平台的wheel,不装了上Linux吧...
# mmcv-full:GPU     mmcv:CPU
%pip install mmcv-full
%pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.8.0/index.html

%pip install mmcv-full=={mmcv_version} -f https://download.openmmlab.com/mmcv/dist/cu115/torch1.11.0/index.html

# 依赖 mmcv, 此处安装一直有问题, 采用手动安装
%pip install mmdet mmsegmentation

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 2.1 MB 568 kB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
Solving environment: / 
  - https://repo.anaconda.com/pkgs/main/linux-64::certifi-2021.10.8-py39h06a4308_0
  - defaults/linux-64::certifi-2021.10.8-py39h06a4308done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [10]:
# 验证
!pip list | grep mm

import mmdet
print(mmdet.__version__)

# !export FORCE_CUDA="1"

mmcls                              0.23.0
mmcv-full                          1.5.2
mmdet                              2.25.0
mmsegmentation                     0.24.1
snowballstemmer                    2.1.0
2.25.0


#### 模型训练

1. 选择,下载模型
2. 准备数据
3. 调用 Python API 构建//训练/推理模型
4. 结果解析,可视化


In [7]:
# mim 也可以用来下载模型,不过这个搜索比较拉
# https://github.com/open-mmlab/mmdetection/tree/master/configs/mask_rcnn
!mim search mmdet --model 'Mask R-CNN'

# mask_rcnn_r50_fpn_2x_coco
#   mask_rcnn 中不同的 backbone (主干网络)
#   r50: 50层ResNet
#   fpn: 区域提议网络
#   2x: learning rate schedule
!mim download mmdet --config mask_rcnn_r50_fpn_2x_coco --dest ./_model

Traceback (most recent call last):
  File "/opt/anaconda/bin/mim", line 8, in <module>
    sys.exit(cli())
  File "/opt/anaconda/lib/python3.9/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/opt/anaconda/lib/python3.9/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/opt/anaconda/lib/python3.9/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/anaconda/lib/python3.9/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/anaconda/lib/python3.9/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/opt/anaconda/lib/python3.9/site-packages/mim/commands/search.py", line 105, in cli
    dataframe = get_model_info(
  File "/opt/anaconda/lib/python3.9/site-packages/mim/commands/search.py", line 169, in get_model_info
    dataframe = conver

In [12]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

config_file = "./_model/mask_rcnn_r50_fpn_2x_coco.py"
checkpoint_file = "./_model/mask_rcnn_r50_fpn_2x_coco_bbox_mAP-0.392__segm_mAP-0.354_20200505_003907-3e542a40.pth"



ImportError: /opt/anaconda/lib/python3.9/site-packages/mmcv/_ext.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZNSt15__exception_ptr13exception_ptr10_M_releaseEv

<a>![分割线](https://fastly.jsdelivr.net/gh/Weidows/Images/img/divider.png)</a>

## 借物表

<a name='cite_note-1' href='#cite_ref-1'>[1]</a>: https://openbayes.com/console/wrh/containers/t93t3LTXlgU

<a name='cite_note-2' href='#cite_ref-2'>[2]</a>: [MMDetection 2.3 安装教程](https://zhuanlan.zhihu.com/p/163645165)

<a name='cite_note-3' href='#cite_ref-3'>[3]</a>: https://mmdetection.readthedocs.io/zh_CN/latest/get_started.html#mmdetection